In [1]:
import os
import cv2
import shutil
import tempfile
import requests
from ultralytics import YOLO

In [2]:
# The URL from which you want to fetch data
base_url = 'http://103.71.191.168/8888/hls/'
playlist_url = f'{base_url}live.stream.m3u8'

# Initialize the YOLO model
model = YOLO('best.pt')
model.fuse()

# Flag to determine if we should exit the main loop
exit_flag = False

# Perform the GET request for the playlist
response = requests.get(playlist_url)

Model summary (fused): 168 layers, 11126358 parameters, 0 gradients


In [3]:
cv2.namedWindow("Inference Streaming", cv2.WINDOW_NORMAL)

if response.status_code == 200:
    # Split the response text into lines
    lines = response.text.strip().split('\n')

    # Extract .ts file URLs from the playlist
    ts_urls = [line for line in lines if line.endswith('.ts')]

    # Process each .ts segment
    for ts_url in ts_urls:
        if exit_flag:  # Check if we need to exit
            break

        full_url = f'{base_url}{ts_url}'

        # Send a GET request for the .ts segment
        ts_response = requests.get(full_url, stream=True)

        if ts_response.status_code == 200:
            # Create a temporary file for the segment
            with tempfile.NamedTemporaryFile(suffix='.ts', delete=False) as tmp_file:
                shutil.copyfileobj(ts_response.raw, tmp_file)
                temp_filename = tmp_file.name

            # Read and display the segment
            cap = cv2.VideoCapture(temp_filename)
            while True:
                ret, frame = cap.read()
                if not ret:
                    break  # Break if there are no more frames in the segment

                # Apply YOLO model on the frame
                results = model(frame, conf=0.26, iou=0.8)
                annotated_image = results[0].plot()

                cv2.namedWindow("Inference Streaming", cv2.WINDOW_FREERATIO)
                cv2.imshow('Inference Streaming', annotated_image)
                if cv2.waitKey(33) & 0xFF == ord('q'):  # Allow exit on 'q' key
                    exit_flag = True
                    break

            cap.release()  # Release the capture object
            os.remove(temp_filename)  # Remove the temporary file

        else:
            print(f"Failed to retrieve segment: {ts_url} (Status code: {ts_response.status_code})")

    if exit_flag:  # Check if we need to exit
        cv2.destroyAllWindows()
else:
    print(f"Failed to retrieve playlist: {response.status_code}")


0: 544x640 (no detections), 447.7ms
Speed: 11.1ms preprocess, 447.7ms inference, 10.0ms postprocess per image at shape (1, 3, 544, 640)

0: 544x640 (no detections), 394.3ms
Speed: 4.2ms preprocess, 394.3ms inference, 0.0ms postprocess per image at shape (1, 3, 544, 640)

0: 544x640 (no detections), 369.9ms
Speed: 3.0ms preprocess, 369.9ms inference, 0.0ms postprocess per image at shape (1, 3, 544, 640)

0: 544x640 (no detections), 318.4ms
Speed: 2.0ms preprocess, 318.4ms inference, 1.0ms postprocess per image at shape (1, 3, 544, 640)

0: 544x640 (no detections), 324.9ms
Speed: 2.0ms preprocess, 324.9ms inference, 0.0ms postprocess per image at shape (1, 3, 544, 640)

0: 544x640 (no detections), 357.8ms
Speed: 3.0ms preprocess, 357.8ms inference, 0.1ms postprocess per image at shape (1, 3, 544, 640)

0: 544x640 (no detections), 330.9ms
Speed: 3.0ms preprocess, 330.9ms inference, 1.0ms postprocess per image at shape (1, 3, 544, 640)

0: 544x640 (no detections), 319.6ms
Speed: 3.0ms pre